In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
import scipy.stats as stats
import sklearn.linear_model as lm
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MultiLabelBinarizer
import sklearn.linear_model as lm
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import  metrics
import ast
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer


In [12]:
df = pd.read_csv('games.csv', index_col=0)
df.head()

,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K


In [13]:
print(df.duplicated().sum())
df = df.drop_duplicates()

382


In [14]:
df = df.drop(['Title', 'Summary', 'Reviews', 'Release Date', 'Team'], axis=1)
df

,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
0,4.5,3.9K,3.9K,"['Adventure', 'RPG']",17K,3.8K,4.6K,4.8K
1,4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",21K,3.2K,6.3K,3.6K
2,4.4,4.3K,4.3K,"['Adventure', 'RPG']",30K,2.5K,5K,2.6K
3,4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",28K,679,4.9K,1.8K
4,4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...
1507,3.2,94,94,"['Adventure', 'Point-and-Click']",763,5,223,67
1508,2.9,264,264,"['Arcade', 'Racing']",1.5K,49,413,107
1509,3.7,210,210,"['Brawler', 'RPG']",1.1K,45,487,206
1510,4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...",269,79,388,602


In [15]:
df['Genres'] = df['Genres'].apply(ast.literal_eval)

def NumTransform(x):
    scale = 1 
    if x[-1] == 'K':
        x = x[:-1]
        scale = 1000
    x = float(x)
    return x * scale

df['Number of Reviews'] = df['Number of Reviews'].apply(NumTransform)
df['Plays'] = df['Plays'].apply(NumTransform)
df['Playing'] = df['Playing'].apply(NumTransform)
df['Backlogs'] = df['Backlogs'].apply(NumTransform)
df['Wishlist'] = df['Wishlist'].apply(NumTransform)
df['Times Listed'] = df['Times Listed'].apply(NumTransform)

df.head()

,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
0,4.5,3900.0,3900.0,"[Adventure, RPG]",17000.0,3800.0,4600.0,4800.0
1,4.3,2900.0,2900.0,"[Adventure, Brawler, Indie, RPG]",21000.0,3200.0,6300.0,3600.0
2,4.4,4300.0,4300.0,"[Adventure, RPG]",30000.0,2500.0,5000.0,2600.0
3,4.2,3500.0,3500.0,"[Adventure, Indie, RPG, Turn Based Strategy]",28000.0,679.0,4900.0,1800.0
4,4.4,3000.0,3000.0,"[Adventure, Indie, Platform]",21000.0,2400.0,8300.0,2300.0


In [16]:
df.isna().sum()

Rating               13
Times Listed          0
Number of Reviews     0
Genres                0
Plays                 0
Playing               0
Backlogs              0
Wishlist              0
dtype: int64

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]
    
class CustomLabelBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, sparse_output=False):
      self.sparse_output = sparse_output
    def fit(self, X, y=None):
      self.enc = MultiLabelBinarizer(sparse_output=self.sparse_output)
      self.enc.fit(x[0] for x in X.values)
      return self
    def transform(self, X, y=None):
      return self.enc.transform(x[0] for x in X.values)


cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Genres"])),
        ("cat_encoder", CustomLabelBinarizer()),
    ])
num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Rating", "Times Listed", "Number of Reviews", "Plays", "Playing", "Backlogs"])),
        ("imputer", SimpleImputer(strategy="mean")),
    ])

from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        #("cat_pipeline", cat_pipeline),
    ])
preprocess_pipeline.fit_transform(df)

array([[4.50e+00, 3.90e+03, 3.90e+03, 1.70e+04, 3.80e+03, 4.60e+03],
       [4.30e+00, 2.90e+03, 2.90e+03, 2.10e+04, 3.20e+03, 6.30e+03],
       [4.40e+00, 4.30e+03, 4.30e+03, 3.00e+04, 2.50e+03, 5.00e+03],
       ...,
       [3.70e+00, 2.10e+02, 2.10e+02, 1.10e+03, 4.50e+01, 4.87e+02],
       [4.10e+00, 1.65e+02, 1.65e+02, 2.69e+02, 7.90e+01, 3.88e+02],
       [2.80e+00, 1.84e+02, 1.84e+02, 1.70e+03, 1.10e+01, 2.39e+02]])

In [24]:
X = df.drop(['Wishlist'], axis=1)
y = df['Wishlist']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
kfold = model_selection.StratifiedKFold(n_splits=5, random_state=0, shuffle=True)


(904, 7) (904,)
(226, 7) (226,)


In [14]:

grid_linear = GridSearchCV(make_pipeline(preprocess_pipeline, PolynomialFeatures(degree=2), linear_model.LinearRegression()),
                    param_grid={'polynomialfeatures__degree': [1, 2, 3]},
                    cv=kfold,
                    refit=True)

grid_linear.fit(X_train, y_train)
print(grid_linear.best_params_)
print("R2: {}".format(metrics.r2_score(y_train, grid_linear.predict(X_train)) ))
print("R2: {}".format(metrics.r2_score(y_test, grid_linear.predict(X_test)) ))

{'polynomialfeatures__degree': 2}
R2: 0.7882149828931213
R2: 0.8083213123569797


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [15]:

grid_lasso = GridSearchCV(make_pipeline(preprocess_pipeline, PolynomialFeatures(degree=2), linear_model.Lasso(alpha=1.0)),
                    param_grid={'polynomialfeatures__degree': [1, 2, 3],
                        'lasso__alpha': [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]},
                    cv=kfold,
                    refit=True)

grid_lasso.fit(X_train, y_train)
print(grid_lasso.best_params_)
print("R2: {}".format(metrics.r2_score(y_train, grid_lasso.predict(X_train)) ))
print("R2: {}".format(metrics.r2_score(y_test, grid_lasso.predict(X_test)) ))

c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.424e+07, tolerance: 3.392e+04
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.391e+07, tolerance: 3.240e+04
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might 

{'lasso__alpha': 100, 'polynomialfeatures__degree': 2}
R2: 0.7881258687562838
R2: 0.8088098697418996


c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.507e+07, tolerance: 4.236e+04
  model = cd_fast.enet_coordinate_descent(


In [16]:

grid_ridge = GridSearchCV(make_pipeline(preprocess_pipeline, PolynomialFeatures(degree=2), linear_model.Ridge(alpha=1.0)),
                    param_grid={'polynomialfeatures__degree': [1, 2, 3],
                        'ridge__alpha': [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]},
                    cv=kfold,
                    refit=True)

grid_ridge.fit(X_train, y_train)
print(grid_ridge.best_params_)
print("R2: {}".format(metrics.r2_score(y_train, grid_ridge.predict(X_train)) ))
print("R2: {}".format(metrics.r2_score(y_test, grid_ridge.predict(X_test)) ))

c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=8.56441e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=9.04318e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=8.40365e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=7.84026e-18): result may not be accurate.
  retur

{'polynomialfeatures__degree': 2, 'ridge__alpha': 10000}
R2: 0.7881696049981158
R2: 0.8079513771913484


In [17]:

grid_elastic = GridSearchCV(make_pipeline(preprocess_pipeline, PolynomialFeatures(degree=2), linear_model.ElasticNet(alpha=1.0)),
                    param_grid={'polynomialfeatures__degree': [1, 2, 3],
                        'elasticnet__alpha': [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001]},
                    cv=kfold,
                    refit=True)

grid_elastic.fit(X_train, y_train)
print(grid_elastic.best_params_)
print("R2: {}".format(metrics.r2_score(y_train, grid_elastic.predict(X_train)) ))
print("R2: {}".format(metrics.r2_score(y_test, grid_elastic.predict(X_test)) ))

c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.218e+07, tolerance: 3.392e+04
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.056e+07, tolerance: 3.240e+04
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might 

{'elasticnet__alpha': 1000, 'polynomialfeatures__degree': 2}
R2: 0.7872477980785622
R2: 0.8080539962402558


In [31]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from xgboost import XGBRegressor



grid_xgb = GridSearchCV(make_pipeline(preprocess_pipeline, PolynomialFeatures(degree=2),
                                       XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)),
                    param_grid={'polynomialfeatures__degree': [1, 2, 3],
                        'xgbregressor__n_estimators': [10000, 100, 10, 1],
                        'xgbregressor__max_depth': [6, 7, 8]},
                    cv=kfold,
                    refit=True)

grid_xgb.fit(X_train, y_train)
print(grid_xgb.best_params_)
print("R2: {}".format(metrics.r2_score(y_train, grid_xgb.predict(X_train)) ))
print("R2: {}".format(metrics.r2_score(y_test, grid_xgb.predict(X_test)) ))


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'polynomialfeatures__degree': 2, 'xgbregressor__max_depth': 6, 'xgbregressor__n_estimators': 100}
R2: 0.9903112290096233
R2: 0.8571137716260337
